# Spell Check Indonesia using Norvig
---
Colab ini menjelaskan implementasi dari dua metode Spell Check yang bisa diterapkan pada bahasa Indonesia yaitu Peter Norvig Spell Checker

## Introduction
Salah satu tahap *text preprocessing* yang berperan untuk meningkatkan performa model *machine learning* adalah *spell checking*. Hal ini disebabkan teks yang terdapat pada internet terutama sosial media seringkali banyak salah tulis (typo), sehingga akan direpresentasikan sebagai kata yang berbeda dan menyebabkan jumlah *vocabulary* kata semakin besar.

Oleh karena itu dengan memperbaiki penulisan kata dari kesalahan dapat meningkatkan performa model.

# load dictionary

Pada langkah ini dilakukan pencarian dan
pengumpulan dataset daftar kata Bahasa Indonesia.
Dataset daftar kata Bahasa Indonesia ini bersumber dari
Kamus Besar Bahasa Indonesia (KBBI). Penulis
menggunakan dataset yang sudah ada yaitu diambil dari
Github Jim GeovedI.


In [2]:
import re
from collections import Counter

# build language model
def words(text): return re.findall(r'\w+', text.lower())

path_corpus = "01-kbbi3-2001-sort-alpha.lst"

WORDS = Counter(words(open(path_corpus).read()))

def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

* 're': Modul untuk operasi dengan ekspresi reguler, seperti pencarian dan manipulasi pola teks.
* 'Counter': Bagian dari modul collections, digunakan untuk menghitung elemen yang dapat dihash dalam sebuah iterable (dalam kasus ini, kata dalam teks).
* Fungsi words mengambil teks sebagai input dan mengembalikan daftar kata-kata dalam teks tersebut.
* re.findall(r'\w+', text.lower()): Menggunakan ekspresi reguler untuk menemukan semua kata (karakter alfanumerik) dalam teks, dan mengubahnya menjadi huruf kecil untuk konsistensi.
* 'path_corpus': Menyimpan jalur file korpus.
* 'WORDS': Membaca isi file korpus, kemudian menghitung frekuensi kemunculan setiap kata dengan menggunakan Counter.
* P: Menghitung probabilitas kemunculan sebuah kata dalam korpus.
* N: Total jumlah kata dalam korpus (dihitung sekali dan disimpan sebagai nilai default).
* WORDS[word] / N: Frekuensi kata dibagi dengan total jumlah kata.

## Peter Norvig Spell Checker

Metode paling sederhana untuk Spelling Corrector adalah dengan metode yang diterapkan oleh [Peter Norvig](https://norvig.com/spell-correct.html) dengan pendekatan Bayes Theorem.

---

Untuk mendapatkan kata yang tepat dapat dituliskan dengan persamaan berikut:

$$correction (w) = argmax_{c ∈ candidates} P(c|w)$$

artinya dicari probabilitas suatu kata $w$ yang paling tinggi dari kumpulan $candidates$.

Misalkan diketahui seperti berikut.
```
w = 'kcing'
candidates = ['kacing', 'kecing', 'kucing']

P('kacing'|'kcing') = 0.6
P('kecing'|'kcing') = 0.3
P('kucing'|'kcing') = 0.8
```

Hasilnya tentu saja untuk `correction('kcing')` adalah `kucing` dengan probabilitas terbesar yaitu `0.8`.


Persamaan di atas dapat dituliskan menjadi

$$correction (w) = argmax_{c ∈ candidates} \frac{P(c)P(w|c)}{P(w)}$$

Karena $P(w)$ adalah sama untuk setiap kemungkinan `candidate c`, maka bisa kita hilangkan dan ditulis menjadi

$$correction (w) = argmax_{c ∈ candidates} P(c)P(w|c)$$

Berdasarkan persamaan di atas, terdapat empat bagian:
1. **Selection Mechanism**: $argmax$, digunakan untuk memilih kandidat yang memiliki probabilitas tertinggi.
2. **Candidate Model**: $c ∈ candidates$, menunjukkan kandidat kata `c` dari kumpulan kandidat `candidates`.
3. **Language Model**: $P(c)$, probabilitas kandidat `c` muncul pada sebuah corpus dokumen. Pada kasus kita, misalkan probabilitas kemunculan `P(kucing)` pada corpus `wiki.txt` adalah `0.3`.
4. **Error Model**: $P(w|c)$, menunjukkan probabilitas bahwa kata `w` adalah teks yang dimaksud pada kandidat `c`. Contohnya `P('kcing'|'kacing') = 0.1` lebih rendah dari `P('kcing'|'kucing') = 0.4`.

In [3]:
# selection mechanism
def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

# candidate model
def candidates(word):
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

# error model
def known(words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

**Correction**
*   correction: Mengembalikan koreksi ejaan yang paling mungkin untuk sebuah kata.
* max(candidates(word), key=P): Memilih kata dari kumpulan kandidat yang memiliki probabilitas tertinggi.
-----
**candidates**
*   candidates: Menghasilkan kandidat koreksi ejaan untuk sebuah kata.
* Urutan pemeriksaan:
1. known([word]): Jika kata tersebut sudah benar.
2. known(edits1(word)): Kata-kata yang berjarak satu edit dari kata tersebut.
3. known(edits2(word)): Kata-kata yang berjarak dua edit dari kata tersebut.
4. [word]: Kata itu sendiri jika tidak ada kandidat lain.
-----
**known**
*  known: Mengembalikan subset dari daftar kata yang terdapat dalam korpus.
*  set(w for w in words if w in WORDS): Mengembalikan hanya kata-kata yang ada dalam WORDS.
---
**edits1**
* edits1: Menghasilkan semua kata yang berjarak satu edit dari kata yang diberikan.
* letters: String berisi semua huruf alfabet.
* splits: Semua cara untuk membagi kata menjadi dua bagian.
* deletes: Menghapus satu karakter.
* transposes: Menukar dua karakter berurutan.
* replaces: Mengganti satu karakter dengan karakter lain.
* inserts: Menyisipkan satu karakter di antara dua karakter.
* set(deletes + transposes + replaces + inserts): Menggabungkan semua hasil edit dan mengembalikan sebagai himpunan.
---
**edits2**
* edits2: Menghasilkan semua kata yang berjarak dua edit dari kata yang diberikan.
* Melakukan dua kali operasi edits1.







------

In [4]:
# contoh perbaikan kata 'kcing'
correction('kcing')

'kaing'

In [5]:
print(f'Banyak kata dari language model: {len(WORDS)}')

Banyak kata dari language model: 61434


In [6]:
known(['kcing'])  # hasilnya kata 'kcing' tidak terdapat pada language model

set()

- Kandidat berikutnya dari `known(edits1('kcing'))`

Terdapat empat operasi pada `edit distance 1`:
- `deletes`: dengan menghapus satu karakter pada kata `kcing` mejadi `cing, kcig, kcin, kcng, king`.
- `transposes`: dengan menggeser satu karakter pada kata `kcing` menjadi `cking, kcign, kcnig, kicng`.
- `replaces`: dengan mengganti satu karakter pada kata `kcing` dengan huruf a-z. Misalkan dengan mengganti huruf `k` pada `kcing` menjadi `acing, bcing, ccing, dcing` dan seterusnya.
- `inserts`: dengan menambahkan satu karakter pada kata `kcing` dengan huruf a-z. Misalkan dengan menambah karakter bagian depan kata `kcing` menjadi `akcing, bkcing, ckcing, dkcing` dan seterusnya.

In [7]:
# cek dari kumpulan candidates yang terdapat pada language model
known(edits1('kcing'))

{'cing', 'kaing', 'king', 'kling', 'kring', 'kucing', 'kuing'}

In [8]:
def P(word, N=sum(WORDS.values())):
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

In [9]:
words = ['cing', 'kaing', 'king', 'kling', 'kring', 'kucing', 'kuing']

# hitung probabilitas
for c in words:
    print(f'P({c}) = {P(c)}')

P(cing) = 1.6277631279096265e-05
P(kaing) = 1.6277631279096265e-05
P(king) = 1.6277631279096265e-05
P(kling) = 1.6277631279096265e-05
P(kring) = 1.6277631279096265e-05
P(kucing) = 1.6277631279096265e-05
P(kuing) = 1.6277631279096265e-05


In [10]:
import time

test_words = ['kcing', 'memkan', 'mrdeka', 'mnyedihkan', 'gimena',
              'terdpt', 'mrmpersulit', 'mhon', 'banos', 'begimana']

In [11]:
# Peter Norvig Spell Checker
start = time.time()
for w in test_words:
    print(f"'{w}' -> '{correction(w)}'")
end = time.time()
print(f'- Peter Norvig: {end-start} detik')
print()

'kcing' -> 'kaing'
'memkan' -> 'memakan'
'mrdeka' -> 'merdeka'
'mnyedihkan' -> 'menyedihkan'
'gimena' -> 'hiena'
'terdpt' -> 'teret'
'mrmpersulit' -> 'mrmpersulit'
'mhon' -> 'mpon'
'banos' -> 'bandos'
'begimana' -> 'begana'
- Peter Norvig: 0.711350679397583 detik

